In [2]:
!pip install tensorflow
!pip install tensorflow_federated

In [3]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow_federated.python.simulation.datasets import download
from tensorflow_federated.python.simulation.datasets import from_tensor_slices_client_data

np.random.seed(0)

2023-02-14 01:40:02.746473: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 01:40:03.741319: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-14 01:40:03.741411: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-14 01:40:03.741421: W tensorfl

In [4]:
"""
Solves the error "RuntimeError: input_dataset: Attempting to capture an EagerTensor without building a function."

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
"""

Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


In [18]:
! pip install kaggle -q

In [19]:
#Download and unzip Kaggle dataset

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download yuweisunut/sidd-segmented-intrusion-detection-dataset
!unzip -q sidd-segmented-intrusion-detection-dataset

sidd-segmented-intrusion-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
replace SIDD/n005/local.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [4]:
NUM_CLIENTS = 15
NUM_EPOCHS = 10
BATCH_SIZE = 32
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

In [5]:
directory = 'SIDD'
client_id = 0
uid = 0
imgs = {}

for client in os.listdir(directory):
  curr_path = f'{directory}/{client}/pcap'

  for subdir in os.listdir(curr_path):
    curr_path = f'{directory}/{client}/pcap/{subdir}/dataset'
    curr_type = subdir[-1:]
   
    for dayscen in os.listdir(curr_path):
      curr_path = f'{directory}/{client}/pcap/{subdir}/dataset/{dayscen}'
      
      for img in os.listdir(curr_path):
        
        if dayscen == 'benign':
            imgs[uid] = {'id': uid, 'client_id': client_id, 'label': str(0), 'fn': img, 'path': curr_path + '/' + img}
        elif dayscen == 'malicious':
            imgs[uid] = {'id': uid, 'client_id': client_id, 'label': str(curr_type), 'fn': img, 'path': curr_path + '/' + img}
        uid +=1
  client_id += 1

In [6]:
img_df = pd.DataFrame.from_dict(imgs,orient='index')
img_df['label'] = img_df['label'].astype(int)
img_df.loc[img_df.index[(img_df['label']==3)],'label'] = 2

In [7]:
print(img_df.head())
print(img_df.shape[0])

   id  client_id  label       fn  \
0   0          0      0  617.jpg   
1   1          0      0   83.jpg   
2   2          0      0  307.jpg   
3   3          0      0   94.jpg   
4   4          0      0  650.jpg   

                                                path  
0  SIDD/n056/pcap/n056_20191015_000001_1/dataset/...  
1  SIDD/n056/pcap/n056_20191015_000001_1/dataset/...  
2  SIDD/n056/pcap/n056_20191015_000001_1/dataset/...  
3  SIDD/n056/pcap/n056_20191015_000001_1/dataset/...  
4  SIDD/n056/pcap/n056_20191015_000001_1/dataset/...  
1385584


In [7]:
def _parse_function(filename, y):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    X = tf.cast(image_decoded, tf.float32)
    return X, y

In [8]:
file_paths = img_df.path
file_labels = img_df["label"]
client_ids = img_df["client_id"]  

    
X = file_paths
y = file_labels
client_ds = tf.data.Dataset.from_tensor_slices((X, y, client_ids))
prepro_client_ds = tf.data.Dataset.from_tensor_slices((X, y))
prepro_client_ds = prepro_client_ds.map(_parse_function)

In [9]:
def preprocess(dataset):

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).prefetch(PREFETCH_BUFFER)

In [10]:
def make_federated_data(client_data, client_ids):
    return [
      preprocess(create_tf_dataset_for_client(client_data, x))
      for x in client_ids
    ]

In [88]:
"""

def create_tf_dataset_for_element(element_id, element_value):
    return tf.data.Dataset.from_tensor_slices((element_id, element_value)).batch(BATCH_SIZE)

def create_federated_dataset_for_client(dataset, id_value):
    filtered_dataset = tff.federated_map(
        lambda x, y, client_id: (x, y, client_id), dataset)
    filtered_dataset = tff.federated_filter(
        lambda x, y, client_id: tf.equal(client_id, id_value),
        filtered_dataset)
    filtered_dataset = tff.federated_map(
        lambda x, y: (x, y), filtered_dataset)

    def load_image(filename, y):
        image_string = tf.io.read_file(filename)
        image_decoded = tf.image.decode_jpeg(image_string, channels=3)
        X = tf.cast(image_decoded, tf.float32)
        return X, y

    filtered_dataset = tff.federated_map(load_image, filtered_dataset)
    filtered_dataset = tff.federated_map(create_tf_dataset_for_element, filtered_dataset)

    return filtered_dataset
"""

In [11]:
def create_tf_dataset_for_client(dataset, id_value):
    # Filter elements with the specified id_value
    
    filtered_dataset = dataset.filter(lambda x, y, client_id: tf.equal(client_id, id_value))

    # Remove the id from the filtered dataset
    filtered_dataset = filtered_dataset.map(lambda x, y, client_id: (x, y))
    
    #Load image from path and convert to float32
    def load_image(filename, y):
        image_string = tf.io.read_file(filename)
        image_decoded = tf.image.decode_jpeg(image_string, channels=3)
        X = tf.cast(image_decoded, tf.float32)
        return X, y

    filtered_dataset = filtered_dataset.map(load_image)
    
    print(type(filtered_dataset))


    return filtered_dataset


In [12]:
clients = img_df["client_id"].unique()
#print(type(client_ds))
federated_train_data = make_federated_data(client_ds, clients)

print(f'Number of client datasets: {len(federated_train_data)}')
print(f'First dataset: {federated_train_data[0]}')

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
Number of client datasets: 15
First dataset: <PrefetchDataset element_spec=(TensorSpec(shape=(None, 

In [21]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = tf.keras.applications.EfficientNetB0(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=3,
    classifier_activation="softmax"
    )

  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [22]:
training_process = tff.learning.algorithms.build_unweighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [23]:
train_state = training_process.initialize()

KeyboardInterrupt: 

In [ ]:
NUM_ROUNDS = 10
for round_num in range(NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

In [ ]:
# Only for documenting training process
logdir = "/tmp/logs/scalars/training/"
try:
  tf.io.gfile.rmtree(logdir)  # delete any previous results
except tf.errors.NotFoundError as e:
  pass # Ignore if the directory didn't previously exist.
summary_writer = tf.summary.create_file_writer(logdir)
train_state = training_process.initialize()

In [ ]:
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    result = training_process.next(train_state, federated_train_data)
    train_state = result.state
    train_metrics = result.metrics
    for name, value in train_metrics['client_work']['train'].items():
      tf.summary.scalar(name, value, step=round_num)

In [ ]:
client_train_datasets = []
client_val_datasets = []

# Initialisieren eines leeren Datasets aus Numpy-Array
test_ds = tf.data.Dataset.from_tensor_slices(np.empty((0, 3)))
shuffle_size = 1000
train_split = 0.8
val_split = 0.1

for client in range(NUM_CLIENTS):
  client_df = pd.DataFrame()
  client_df = img_df.loc[img_df['client_id'] == client]
  #print(client_df.head())
  file_paths = img_df.path
  file_labels = img_df["label"]
  X = file_paths
  y = file_labels  
  client_ds = tf.data.Dataset.from_tensor_slices((X, y))
  client_ds = client_ds.map(_parse_function) 
  client_ds = client_ds.shuffle(shuffle_size, seed=12)
  client_ds = client_ds.batch(32)


  ds_size = client_ds.cardinality().numpy()
  train_size = int(train_split * ds_size)
  val_size = int(val_split * ds_size)



  train_ds = client_ds.take(train_size)
  print(train_ds)
  val_ds = client_ds.skip(train_size).take(val_size)
  test_ds = client_ds.skip(train_size + val_size)
  print(client_ds)
    
  client_train_datasets.append(train_ds)
  client_val_datasets.append(train_ds)

In [ ]:
!ls {logdir}
%tensorboard --logdir {logdir} --port=0

In [37]:
client_df = pd.DataFrame()
client_df = img_df
file_paths = img_df.path
file_labels = img_df["label"]
client_ids = img_df["client_id"]  

    
X = file_paths
y = file_labels
client_ds = tff.simulation.from_tensor_slices((X, y, client_ids))
#client_ds = SIDDClientDataset(X, y, client_ids)
prepro_client_ds = tff.simulation.from_tensor_slices((X, y))
prepro_client_ds = prepro_client_ds.map(_parse_function)
#print(client_ds.element_spec)
    
#client_ds = client_ds.map(_parse_function)
#print(client_ds.element_spec)

for i, (X_batch, y_batch, client_id_batch) in enumerate(client_ds.take(5)):
    print(f"Batch {i}: X_batch.shape={X_batch.shape}, y_batch.shape={y_batch.shape}, client_id_batch.shape={client_id_batch.shape}")

try:
    client_ids = client_ds.client_ids
    print("The client_ds object has the property 'client_ids'")
except AttributeError:
    print("The client_ds object does not have the property 'client_ids'")

AttributeError: module 'tensorflow_federated.python.simulation' has no attribute 'from_tensor_slices'